In [10]:
import cv2
from ultralytics import YOLO
from PIL import Image
import cvzone
import os
import numpy as np
import sort

# YOLO

# implement yolo to video frame

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [22]:
import wget
wget.download("https://raw.githubusercontent.com/abewley/sort/master/sort.py")

'sort.py'

In [11]:
video_path = ".\Video\TrafficFlow.mp4"
image_path = './Image/imageFilter.png'

In [12]:
model = YOLO('./model/yolov8n.pt')

In [6]:
model.export(format='onnx')

Ultralytics YOLOv8.1.2 🚀 Python-3.11.8 torch-2.1.2+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'model\yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.4/14.4 MB 195.5 kB/s eta 0:00:00
   -------------------------------------- 420.9/420.9 kB 164.3 kB/s eta 0:00:00

requirements: AutoUpdate success ✅ 102.0s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 108.0s, saved as 'model\yolov8n.onnx' (12.2 MB)

Export complete (112.2s)
Results saved to F:\python\EveryDayCoding\ComputerVision\ObjectDetection\YOLO\CarCount\model
Predict:         yolo predict task=detect model=

'model\\yolov8n.onnx'

In [9]:
model = YOLO('./model/yolov8n.onnx')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [13]:
wCam, hCam = 640, 480

image_overlay = cv2.imread(image_path)
# image_overlay = cv2.resize(image_overlay, (wCam, hCam))

capture = cv2.VideoCapture(video_path)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, wCam)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, hCam)
pTime = 0
object_name = ["car", "truck", "bus", "motorbike"]

tracker = sort.Sort(max_age=20, min_hits=3, iou_threshold=0.2)
totalCount = []

line = (100, hCam//2, 600, hCam//2)
while (capture.isOpened()):
    ret, frame = capture.read() 
    image_overlay = cv2.resize(image_overlay, (frame.shape[1], frame.shape[0]))
    frame_overlay = cv2.bitwise_and(image_overlay, frame)
    results = model(frame_overlay)[0]
    object_info_detected = np.empty((0, 5))
    
    if ret == True: 
        # yolo
        for box in results.boxes:
            class_name = results.names[box.cls[0].item()] # Class name
            cords = box.xyxy[0].tolist()
            x1, y1, x2, y2 = (round(x) for x in cords)
            x, y, w, h = x1, y1, x2 - x1, y2 - y1    
            confidence = round(box.conf[0].item(), 2)
            print(confidence)
            # cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),3)
            
            # cvzone.cornerRect(
            #                     frame,  # The image to draw on
            #                     (x, y, w, h ),  # The position and dimensions of the rectangle (x, y, width, height)
            #                     l=30,  # Length of the corner edges
            #                     t=2,  # Thickness of the corner edges
            #                     rt=1,  # Thickness of the rectangle
            #                     colorR=(255, 0, 255),  # Color of the rectangle
            #                     colorC=(0, 255, 0)  # Color of the corner edges
            #                 )
            
            
            # cvzone.putTextRect(
            #                     frame, f"{confidence} {class_name}", (max(0, x), max(30, y)),  # Image and starting position of the rectangle
            #                     scale=1, thickness=1,  # Font scale and thickness
            #                     colorT=(0, 0, 255), colorR=(255, 100, 0),  # Text color and Rectangle color
            #                     font=cv2.FONT_HERSHEY_PLAIN,  # Font type
            #                     offset=1,  # Offset of text inside the rectangle
            #                     border=1, colorB=(0, 5, 0)  # Border thickness and color
            #                 )
            
            if class_name in object_name and confidence > 0.5:
                object_info_array = np.array([x1, y1, x2, y2, confidence])
                object_info_detected = np.vstack((object_info_detected, object_info_array))
        
        resultTracker = tracker.update(object_info_detected)
        cv2.line(frame, (line[0], line[1]), (line[2], line[3]), (0, 0, 255), 3)
        
        for result_track in resultTracker:
            x1, y1, x2, y2, id = result_track
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            x, y, w, h = x1, y1, x2 - x1, y2 - y1  
            
            cvzone.cornerRect(
                    frame,  # The image to draw on
                    (x, y, w, h ),  # The position and dimensions of the rectangle (x, y, width, height)
                    l=30,  # Length of the corner edges
                    t=2,  # Thickness of the corner edges
                    rt=1,  # Thickness of the rectangle
                    colorR=(255, 0, 255),  # Color of the rectangle
                    colorC=(0, 255, 0)  # Color of the corner edges
                )
            
            cvzone.putTextRect(
                                frame, f"{id}", (max(0, x), max(30, y)),  # Image and starting position of the rectangle
                                scale=1, thickness=1,  # Font scale and thickness
                                colorT=(0, 0, 255), colorR=(255, 100, 0),  # Text color and Rectangle color
                                font=cv2.FONT_HERSHEY_PLAIN,  # Font type
                                offset=1,  # Offset of text inside the rectangle
                                border=1, colorB=(0, 5, 0)  # Border thickness and color
                            )
            
            centerX, centerY = x1 + w//2, y1 + h//2
            
            cv2.circle(frame, (centerX, centerY), 4, (0, 225, 255), cv2.FILLED)
            
            if line[0] < centerX < line[2] and line[1] - 5 < centerY < line[3]:
                if totalCount.count(id) == 0:
                    totalCount.append(id)
                    cv2.line(frame, (line[0], line[1]), (line[2], line[3]), (255, 0, 0), 3)
            
            cvzone.putTextRect(
                    frame, f"Count viachle passed : {len(totalCount)}", (int(wCam//6), int(hCam//10)),  # Image and starting position of the rectangle
                    scale=1, thickness=1,  # Font scale and thickness
                    colorT=(0, 0, 255), colorR=(255, 100, 0),  # Text color and Rectangle color
                    font=cv2.FONT_HERSHEY_SIMPLEX,  # Font type
                    offset=2,  # Offset of text inside the rectangle
                    border=1, colorB=(0, 5, 0)  # Border thickness and color
                )
            
        # frame = cv2.resize(frame, (640, 640), 
        #        interpolation = cv2.INTER_LINEAR)
        cv2.imshow('Frame', frame) 
        
        if cv2.waitKey(25) in (ord('q'), ord('Q')):
            break
    else: 
        break
  
# After the loop release the capture object 
capture.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


0: 320x640 1 htyellow, 1231.1ms
Speed: 16.9ms preprocess, 1231.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
0.73

0: 320x640 1 htyellow, 995.3ms
Speed: 4.0ms preprocess, 995.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0.76

0: 320x640 1 htyellow, 709.5ms
Speed: 7.5ms preprocess, 709.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
0.78

0: 320x640 1 htyellow, 904.4ms
Speed: 6.0ms preprocess, 904.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
0.82

0: 320x640 1 htyellow, 940.2ms
Speed: 4.0ms preprocess, 940.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
0.83

0: 320x640 1 htyellow, 713.4ms
Speed: 4.0ms preprocess, 713.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
0.83

0: 320x640 1 htyellow, 1 htwhite, 936.1ms
Speed: 4.0ms preprocess, 936.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
0.83
0.33

0: 320x640 1 htyellow, 1 htwhite, 8

In [8]:
import typing

In [46]:
class Object_Detection:
    def __init__(self, model:str, width:int, height:int, object_name:typing.List[str] = ["car", "truck", "bus", "motorbike"]):
        self.object_name = object_name
        self.model = YOLO(model)
        self.width = width
        self.height = height
        self.object_name = object_name
        self.frame = None
        

    # set image
    def set_image(self, image_path:str):
        image_overlay = cv2.imread(image_path)
        image_overlay = cv2.resize(image_overlay, (self.width, self.height))
        return image_overlay
    
    
    # draw reactangle
    def set_reactangle(self, 
                       x1, y1, x2, y2, 
                       length, 
                       thickness_edges, 
                       thickness_rectangle, 
                       color_rectangle:typing.Tuple[int:"Blue", int:"Green", int:"Red"], 
                       color_corners:typing.Tuple[int:"Blue", int:"Green", int:"Red"]):
        
        """
        description: Draw reactangle on frame
                x1, y1, x2, y2: position and dimensions of the rectangle 
                length: length of the edges
                thickness_edges: thickness of the edges
                thickness_rectangle: thickness of the rectangle
                color_rectangle: color of the rectangle
                color_corners: color of the corners

        """
        
        cvzone.cornerRect(
                self.frame, 
                (x1, y1, x2 - x1, y2 - y1), 
                l=length, 
                t=thickness_edges, 
                rt=thickness_rectangle, 
                colorR=color_rectangle, 
                colorC=color_corners 
            )
        
    
    # draw text
    def set_Text(self, 
                 text:str, 
                 font_scale:int, 
                 thickness:int,
                 x:int, y:int, 
                 offset:int, 
                 border:int,
                 color_text:typing.Tuple[int:"Blue", int:"Green", int:"Red"],
                 color_rectangle:typing.Tuple[int:"Blue", int:"Green", int:"Red"],
                 color_border:typing.Tuple[int:"Blue", int:"Green", int:"Red"]
                 ):
        
        """
        description: Draw text on frame
                text: text
                font_scale: font scale
                thickness: thickness
                x, y: position
                offset: offset
                border: border
                color_text: color of the text
                color_rectangle: color of the rectangle
                color_border: color of the border
        """

        cvzone.putTextRect(
                self.frame, text, (x, y), 
                scale=font_scale, thickness=thickness,  
                colorT=color_text, colorR=color_rectangle,  
                font=cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 
                offset=offset,  
                border=border, colorB=color_border 
            )
        

    # get inference
    def _get_inference(self, image):
        results = self.model(image)[0]
        return results
    
    
    # get box
    def get_box(self, results):
        for box in results.boxes:
            class_name = results.names[box.cls[0].item()] # Class name
            cords = box.xyxy[0].tolist()
            x1, y1, x2, y2 = (round(x) for x in cords)
            # x, y, w, h = x1, y1, x2 - x1, y2 - y1    
            confidence = round(box.conf[0].item(), 2)
        
            self.set_reactangle(
                                x1, y1, x2, y2, 
                                length=30, 
                                thickness_edges=2, 
                                thickness_rectangle=1, 
                                color_rectangle=(255, 0, 255), 
                                color_corners=(0, 255, 0) 
                                )
    
            
    def run_video(self, video_path:str):
        capture = cv2.VideoCapture(video_path)
        capture.set(3, self.width)
        capture.set(4, self.height)
        
        while (capture.isOpened()):
            ret, frame = capture.read() 
            self.frame = frame
            results = self._get_inference(frame)
            # results = model(frame)[0]
               
            if ret == True:
                self.get_box(results)
                
                if cv2.waitKey(25) in (ord('q'), ord('Q')):
                    break
            else: 
                break
            
            cv2.imshow('Frame', frame) 
        
        capture.release() 
        cv2.destroyAllWindows()
            

    
    

## the OOP havent done yet, just look the scratch code for reference

In [47]:
OD = Object_Detection('./model/yolov8n.pt', 640, 480, ["car", "truck", "bus", "motorbike"])
OD.run_video(video_path='.\Video\TrafficFlow.mp4')


0: 384x640 20 cars, 1 bus, 2 trucks, 249.5ms
Speed: 0.0ms preprocess, 249.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 2 trucks, 223.1ms
Speed: 4.0ms preprocess, 223.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 138.2ms
Speed: 3.4ms preprocess, 138.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 146.9ms
Speed: 0.0ms preprocess, 146.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 1 truck, 135.9ms
Speed: 0.0ms preprocess, 135.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 129.6ms
Speed: 2.0ms preprocess, 129.6ms inference, 13.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 149.7ms
Speed: 2.0ms preprocess, 149.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384